In [ ]:
# Exploratory Data Analysis (EDA)

This notebook contains the initial exploration of the healthcare analytics dataset.

## 1. Load Data


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown


In [9]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load all datasets
app_usage = pd.read_csv('../data/app_usage.csv')
churn_labels = pd.read_csv('../data/churn_labels.csv')
claims = pd.read_csv('../data/claims.csv')
web_visits = pd.read_csv('../data/web_visits.csv')

# Convert date columns
churn_labels['signup_date'] = pd.to_datetime(churn_labels['signup_date'])
claims['diagnosis_date'] = pd.to_datetime(claims['diagnosis_date'])
app_usage['timestamp'] = pd.to_datetime(app_usage['timestamp'])
web_visits['timestamp'] = pd.to_datetime(web_visits['timestamp'])

display(Markdown("**Data loaded successfully!**"))
display(Markdown(f"**Dataset shapes:**"))
display(Markdown(f"  - app_usage: {app_usage.shape}"))
display(Markdown(f"  - churn_labels: {churn_labels.shape}"))
display(Markdown(f"  - claims: {claims.shape}"))
display(Markdown(f"  - web_visits: {web_visits.shape}"))


**Data loaded successfully!**

**Dataset shapes:**

  - app_usage: (97789, 3)

  - churn_labels: (10000, 4)

  - claims: (64576, 3)

  - web_visits: (259340, 5)

## 2. Merge Process

Merge all datasets into a unified member-level dataset for analysis.


In [ ]:
# Start with churn_labels as the base (primary table)
merged_data = churn_labels.copy()

display(Markdown("**Starting with churn_labels as base table**"))
display(Markdown(f"Base shape: {merged_data.shape}"))


In [ ]:
# Aggregate app_usage to member level
display(Markdown("### Aggregating App Usage Data"))

app_usage_agg = app_usage.groupby('member_id').agg({
    'timestamp': [
        ('app_sessions_count', 'count'),
        ('app_first_session', 'min'),
        ('app_last_session', 'max'),
        ('app_days_active', lambda x: (x.max() - x.min()).days if len(x) > 1 else 0)
    ]
}).reset_index()

# Flatten column names
app_usage_agg.columns = ['member_id', 'app_sessions_count', 'app_first_session', 'app_last_session', 'app_days_active']

# Calculate days since last session (if applicable)
app_usage_agg['app_days_since_last_session'] = (
    (app_usage['timestamp'].max() - app_usage_agg['app_last_session']).dt.days
)

display(Markdown(f"App usage aggregated shape: {app_usage_agg.shape}"))
display(app_usage_agg.head())


In [15]:
## App usage

In [10]:
# Display basic information about each dataset
display(Markdown("## APP USAGE DATA"))
display(app_usage.info())
display(Markdown("**First few rows:**"))
display(app_usage.head(10))
display(Markdown("**Basic statistics:**"))
display(app_usage.describe(include='all'))


## APP USAGE DATA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97789 entries, 0 to 97788
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   member_id   97789 non-null  int64         
 1   event_type  97789 non-null  object        
 2   timestamp   97789 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.2+ MB


None

**First few rows:**

,member_id,event_type,timestamp
0,1,session,2025-07-13 08:43:37
1,1,session,2025-07-14 15:28:58
2,1,session,2025-07-02 19:43:17
3,1,session,2025-07-09 09:40:14
4,1,session,2025-07-07 00:39:54
5,1,session,2025-07-03 04:40:05
6,2,session,2025-07-10 11:02:06
7,2,session,2025-07-08 19:04:18
8,2,session,2025-07-02 18:04:43
9,2,session,2025-07-12 16:58:17


**Basic statistics:**

,member_id,event_type,timestamp
count,97789.000000,97789,97789
unique,NaN,1,NaN
top,NaN,session,NaN
freq,NaN,97789,NaN
mean,4993.659727,NaN,2025-07-07 23:56:58.643231744
min,1.000000,NaN,2025-07-01 00:00:00
25%,2508.000000,NaN,2025-07-04 11:35:32
50%,4984.000000,NaN,2025-07-08 00:12:43
75%,7502.000000,NaN,2025-07-11 11:33:50
max,10000.000000,NaN,2025-07-14 23:59:51


In [11]:
display(Markdown("## CHURN LABELS DATA"))
display(churn_labels.info())
display(Markdown("**First few rows:**"))
display(churn_labels.head(10))
display(Markdown("**Basic statistics:**"))
display(churn_labels.describe(include='all'))
display(Markdown("**Churn distribution:**"))
display(churn_labels['churn'].value_counts())
display(Markdown("**Outreach distribution:**"))
display(churn_labels['outreach'].value_counts())
display(Markdown("**Churn by Outreach:**"))
display(pd.crosstab(churn_labels['churn'], churn_labels['outreach'], margins=True))


## CHURN LABELS DATA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   member_id    10000 non-null  int64         
 1   signup_date  10000 non-null  datetime64[ns]
 2   churn        10000 non-null  int64         
 3   outreach     10000 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 312.6 KB


None

**First few rows:**

,member_id,signup_date,churn,outreach
0,1,2024-04-12,0,0
1,2,2025-03-11,0,0
2,3,2024-09-27,0,0
3,4,2024-04-16,1,0
4,5,2024-03-12,0,1
5,6,2024-01-21,0,0
6,7,2024-05-01,0,1
7,8,2025-04-11,0,1
8,9,2024-08-02,1,1
9,10,2024-11-26,0,0


...

churn
0    7979
1    2021
Name: count, dtype: int64

**Outreach distribution:**

outreach
0    6016
1    3984
Name: count, dtype: int64

**Churn by Outreach:**

outreach,0,1,All
churn,,,
0,4768,3211,7979
1,1248,773,2021
All,6016,3984,10000


In [12]:
display(Markdown("## CLAIMS DATA"))
display(claims.info())
display(Markdown("**First few rows:**"))
display(claims.head(10))
display(Markdown("**Basic statistics:**"))
display(claims.describe(include='all'))
display(Markdown(f"**Unique ICD codes count:** {claims['icd_code'].nunique()}"))
display(Markdown("**Top 10 most common ICD codes:**"))
display(claims['icd_code'].value_counts().head(10))


## CLAIMS DATA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64576 entries, 0 to 64575
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   member_id       64576 non-null  int64         
 1   icd_code        64576 non-null  object        
 2   diagnosis_date  64576 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.5+ MB


None

**First few rows:**

,member_id,icd_code,diagnosis_date
0,1,Z71.3,2025-07-09
1,1,Z71.3,2025-07-14
2,1,J00,2025-07-12
3,1,M54.5,2025-07-01
4,1,I10,2025-07-12
5,1,J00,2025-07-01
6,1,J00,2025-07-07
7,1,J00,2025-07-13
8,1,E11.9,2025-07-09
9,2,E11.9,2025-07-08


**Basic statistics:**

,member_id,icd_code,diagnosis_date
count,64576.000000,64576,64576
unique,NaN,10,NaN
top,NaN,Z71.3,NaN
freq,NaN,9145,NaN
mean,5015.547479,NaN,2025-07-07 11:38:27.532209920
min,1.000000,NaN,2025-07-01 00:00:00
25%,2484.000000,NaN,2025-07-04 00:00:00
50%,5036.000000,NaN,2025-07-07 00:00:00
75%,7538.000000,NaN,2025-07-11 00:00:00
max,10000.000000,NaN,2025-07-14 00:00:00


**Unique ICD codes count:** 10

**Top 10 most common ICD codes:**

icd_code
Z71.3    9145
I10      9006
E11.9    8961
H10.9    5414
B34.9    5410
A09      5387
M54.5    5385
J00      5346
R51      5277
K21.9    5245
Name: count, dtype: int64

In [13]:
display(Markdown("## WEB VISITS DATA"))
display(web_visits.info())
display(Markdown("**First few rows:**"))
display(web_visits.head(10))
display(Markdown("**Basic statistics:**"))
display(web_visits.describe(include='all'))
display(Markdown(f"**Unique URLs count:** {web_visits['url'].nunique()}"))
display(Markdown("**Top 10 most visited URLs:**"))
display(web_visits['url'].value_counts().head(10))


## WEB VISITS DATA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259340 entries, 0 to 259339
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   member_id    259340 non-null  int64         
 1   url          259340 non-null  object        
 2   title        259340 non-null  object        
 3   description  259340 non-null  object        
 4   timestamp    259340 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 9.9+ MB


None

**First few rows:**

,member_id,url,title,description,timestamp
0,1,https://health.wellco/chronic/859,Diabetes management,Blood sugar and glycemic control,2025-07-02 22:38:22
1,1,https://portal.site/tech/328,Gadget roundup,Smartphones and laptops news,2025-07-02 11:30:47
2,1,https://health.wellco/heart/792,Hypertension basics,Blood pressure and lifestyle changes,2025-07-14 00:38:39
3,2,https://example.com/gaming/674,Game reviews,Strategy tips and updates,2025-07-07 02:56:06
4,2,https://living.better/stress/325,Stress reduction,Mindfulness and wellness,2025-07-02 15:53:38
5,2,https://care.portal/sleep/928,Restorative sleep tips,Sleep apnea screening and hygiene,2025-07-02 13:41:14
6,2,https://guide.wellness/wellness/814,Healthy eating guide,Tips on nutrition and balanced diets,2025-07-01 00:37:35
7,2,https://care.portal/aerobic/384,Aerobic exercise,Cardiovascular fitness and endurance,2025-07-07 14:04:09
8,2,https://guide.wellness/heart/881,Hypertension basics,Blood pressure and lifestyle changes,2025-07-04 16:29:43
9,2,https://health.wellco/diabetes/194,HbA1c targets,Improving glycemic control and blood glucose,2025-07-01 13:19:22


**Basic statistics:**

,member_id,url,title,description,timestamp
count,259340.000000,259340,259340,259340,259340
unique,NaN,69894,26,26,NaN
top,NaN,https://media.hub/movies/434,Top destinations,City guides and itineraries,NaN
freq,NaN,18,22600,22600,NaN
mean,5031.441216,NaN,NaN,NaN,2025-07-08 00:15:29.811907072
min,1.000000,NaN,NaN,NaN,2025-07-01 00:00:01
25%,2529.000000,NaN,NaN,NaN,2025-07-04 12:34:04.750000128
50%,5034.000000,NaN,NaN,NaN,2025-07-08 00:25:37.500000
75%,7550.000000,NaN,NaN,NaN,2025-07-11 11:55:17
max,10000.000000,NaN,NaN,NaN,2025-07-14 23:59:49


**Unique URLs count:** 69894

**Top 10 most visited URLs:**

url
https://media.hub/movies/434      18
https://media.hub/cars/275        18
https://example.com/gaming/283    18
https://portal.site/gaming/682    18
https://world.news/movies/120     17
https://media.hub/travel/563      17
https://portal.site/movies/614    17
https://portal.site/gaming/338    17
https://world.news/sports/764     17
https://media.hub/sports/269      17
Name: count, dtype: int64

## 3. Data Quality Checks


In [14]:
# Check for missing values
display(Markdown("### Missing values in each dataset:"))
display(Markdown("**App Usage:**"))
display(app_usage.isnull().sum())
display(Markdown("**Churn Labels:**"))
display(churn_labels.isnull().sum())
display(Markdown("**Claims:**"))
display(claims.isnull().sum())
display(Markdown("**Web Visits:**"))
display(web_visits.isnull().sum())

# Check for duplicate rows
display(Markdown("### Duplicate rows check:"))
display(Markdown(f"- App Usage duplicates: {app_usage.duplicated().sum()}"))
display(Markdown(f"- Churn Labels duplicates: {churn_labels.duplicated().sum()}"))
display(Markdown(f"- Claims duplicates: {claims.duplicated().sum()}"))
display(Markdown(f"- Web Visits duplicates: {web_visits.duplicated().sum()}"))

# Check unique member counts
display(Markdown("### Unique member counts:"))
display(Markdown(f"- App Usage unique members: {app_usage['member_id'].nunique()}"))
display(Markdown(f"- Churn Labels unique members: {churn_labels['member_id'].nunique()}"))
display(Markdown(f"- Claims unique members: {claims['member_id'].nunique()}"))
display(Markdown(f"- Web Visits unique members: {web_visits['member_id'].nunique()}"))


### Missing values in each dataset:

**App Usage:**

member_id     0
event_type    0
timestamp     0
dtype: int64

**Churn Labels:**

member_id      0
signup_date    0
churn          0
outreach       0
dtype: int64

...

### Unique member counts:

- App Usage unique members: 9998

- Churn Labels unique members: 10000

- Claims unique members: 9980

- Web Visits unique members: 9975